In [ ]:
!pip install kaggle

In [ ]:
#configurating the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d girinathrs211555/aadhaar-datasets

 95% 353M/370M [00:05<00:00, 85.0MB/s]
100% 370M/370M [00:05<00:00, 74.4MB/s]


In [ ]:
# extrcing the compossed Dataset
from zipfile import ZipFile
dataset = '/content/aadhaar-datasets.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
Aadhaar_card_files = os.listdir('/content/Aaadhaar AND Non aadhaar/Aadhaar')
print(Aadhaar_card_files[0:5])

print(Aadhaar_card_files[-5:])

['Aadhaar_card (2475).jpg', 'Aadhaar_card (52).jpg', 'Aadhaar_card (1715).jpg', 'Aadhaar_card (3385).jpg', 'Aadhaar_card (2635).jpg']
['Aadhaar_card (138).jpg', 'Aadhaar_card (2266).jpg', 'Aadhaar_card (3501).jpg', 'Aadhaar_card (3356).jpg', 'Aadhaar_card (4094).jpg']


In [ ]:
Non_aadhaar_card_files = os.listdir('/content/Aaadhaar AND Non aadhaar/Non Aadhaar')
print(Non_aadhaar_card_files[0:5])

print(Non_aadhaar_card_files[-5:])

['Non_Aadhaar  (3236).jpg', 'Non_Aadhaar  (1631).jpg', 'Non_Aadhaar  (709).jfif', 'Non_Aadhaar  (2279).jpg', 'Non_Aadhaar  (626).jpg']
['Non_Aadhaar  (1814).jpg', 'Non_Aadhaar  (598).jfif', 'Non_Aadhaar  (1013).jpg', 'Non_Aadhaar  (42).jfif', 'Non_Aadhaar  (743).jfif']


In [ ]:
print('Number of with Aadhaar_card_files:', len(Aadhaar_card_files))
print('Number of without Non_aadhaar_card_files:', len(Non_aadhaar_card_files))

Number of with Aadhaar_card_files: 5009
Number of without Non_aadhaar_card_files: 5021


In [ ]:
#create the labels

Aadhaar_card_labels = [1]*5009

Non_aadhaar_card_labels = [0]*5021

In [ ]:
# Checking wheather the labels are created as expected
print(Aadhaar_card_labels[0:5])

print(Non_aadhaar_card_labels[0:5])

[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0]


In [ ]:
# I am Concanating  the  two class as labels

labels = Aadhaar_card_labels + Non_aadhaar_card_labels

print(len(labels))
print(labels[0:5])
print(labels[-5:])

10030
[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0]


In [ ]:
# Convert images to numpy arrays

Aadhaar_card__path = '/content/Aaadhaar AND Non aadhaar/Aadhaar/'

data = []
for img_file in Aadhaar_card_files:

  image = Image.open(Aadhaar_card__path + img_file)
  image = image.resize((128,128))
  image = image.convert("RGB")
  image = np.array(image)
  data.append(image)


Non_aadhaar_card_path = '/content/Aaadhaar AND Non aadhaar/Non Aadhaar/'

for img_file in Non_aadhaar_card_files:

  image = Image.open(Non_aadhaar_card_path + img_file)
  image = image.resize((128,128))
  image = image.convert("RGB")
  image = np.array(image)
  data.append(image)

In [ ]:
# Coverting image list and the label list to numpy arrays

X = np.array(data)
Y = np.array(labels)

In [ ]:
type(X)

numpy.ndarray

In [ ]:
type(Y)

numpy.ndarray

In [ ]:
print(X.shape)
print(Y.shape)

(10030, 128, 128, 3)
(10030,)


In [ ]:
print(Y)

[1 1 1 ... 0 0 0]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
#Scaling the data

X_train_scaled = X_train/255

X_test_scaled = X_test/255

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
num_of_classes = 2

model = keras.Sequential()

model.add(keras.layers.Conv2D(300, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(600, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(800, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_of_classes, activation='sigmoid'))

In [ ]:
# Compile the neural network

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
#traing the neural network
history = model.fit(X_train_scaled, Y_train, validation_split=0.2, epochs=5)

Epoch 1/5
 32/201 [===>..........................] - ETA: 1:45:56 - loss: 1.0779 - acc: 0.5195

In [ ]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy=', accuracy)

In [ ]:
h = history

#plot the loss value
plt.plot(h.history['loss'], label='train loss')
plt.plot(h.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

#plot the accuracy value
plt.plot(h.history['acc'], label='train accuracy')
plt.plot(h.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
input_image_path = input('Path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resized = cv2.resize(input_image, (128, 128))

input_image_scaled = input_image_resized/255

input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])

input_prediction = model.predict(input_image_reshaped)

print(input_prediction)

input_pred_label = np.argmax(input_prediction)

print(input_pred_label)

if input_pred_label == 1:

  print('This is an aadhaar card')

else:

  print('This not an aadhaar card')
